# This notebook tests the Selection class. 

In [1]:
import pathlib

import astropy.units as u
from astropy.time import Time

import dysh
from dysh import util
from dysh.fits.gbtfitsload import GBTFITSLoad
from dysh.util.selection import Selection


In [2]:
file = util.get_project_testdata() / "TGBT21A_501_11/testselection.fits"
sdf = GBTFITSLoad(file)
sdf.summary()

,SCAN,OBJECT,VELOCITY,PROC,PROCSEQN,RESTFREQ,DOPFREQ,# IF,# POL,# INT,# FEED,AZIMUTH,ELEVATIO
0,144,PSR_B0329+54,0.0,Track,1,1.617185,1.420406,5,2,2,1,326.831344,18.044293
1,177,ScoX-1,0.0,OnOff,1,6.25,4.5,1,2,1,1,127.172148,17.088829
2,178,ScoX-1,0.0,OnOff,2,4.607143,4.5,2,2,1,1,125.678986,15.850586
3,152,NGC2415,3784.0,OnOff,1,1.570635,1.420406,3,2,1,1,286.483603,41.161583
4,155,3C196,0.0,OnOff,2,1.570635,1.420406,3,2,1,1,303.968027,49.164103
5,156,NGC2782,2543.139777,OnOff,1,1.570635,1.420406,3,2,1,1,286.217653,58.03076


### Create a selection object for SDFITS data.

In [3]:
s = Selection(sdf)

### Select an object and polarization number.

In [4]:
s.select(object="NGC2415", plnum=0)

### No tag was given so a unique random tag is generated.

In [5]:
s.show()

 ID    TAG     OBJECT PLNUM # SELECTED
--- --------- ------- ----- ----------
  0 6a6b2e9ff NGC2415     0          5


### Add a rule selection IF numbers, and give it a tag

In [6]:
s.select(ifnum=[0, 2], tag="ifnums")

In [7]:
s.show()

 ID    TAG     OBJECT IFNUM  PLNUM # SELECTED
--- --------- ------- ------ ----- ----------
  0 6a6b2e9ff NGC2415            0          5
  1    ifnums         [0, 2]               26


### Now combine these to a final selection which ORs the rows of the above table.
There should be 3 rows in the final selection

In [8]:
s.final

,OBJECT,BANDWID,DATE-OBS,DURATION,EXPOSURE,TSYS,TDIM7,TUNIT7,CTYPE1,CRVAL1,...,FDNUM,INT,NSAVE,HDU,BINTABLE,ROW,PROC,_OBSTYPE,_SUBOBSMODE,UTC
0,NGC2415,23437500.0,2021-02-10T07:43:36.97,0.998244,0.487937,1.0,"(32768,1,1,1)",Counts,FREQ-OBS,1.402545e+09,...,0,150,-1,1,0,11,OnOff,PSWITCHON,TPWCAL,2021-02-10T07:38:37.000
1,NGC2415,23437500.0,2021-02-10T07:43:36.97,0.998244,0.040545,1.0,"(32768,1,1,1)",Counts,FREQ-OBS,1.402545e+09,...,0,150,-1,1,0,12,OnOff,PSWITCHON,TPWCAL,2021-02-10T07:38:37.000
2,NGC2415,23437500.0,2021-02-10T07:43:36.97,0.998244,0.487937,1.0,"(32768,1,1,1)",Counts,FREQ-OBS,1.647540e+09,...,0,150,-1,1,0,19,OnOff,PSWITCHON,TPWCAL,2021-02-10T07:38:37.000


### Remove these selections
This can be done by id or tag.  Multiple rows with the same tag will all be removed.

In [9]:
s.remove(id=0)
s.remove(tag='ifnums')

### Select by range.
Lower limits are give by (value,None) or (value,).  Upper limits are given by (None,value), since (,value) is not valid python.  For coordinates the default unit is taken to be degrees. Other units can be explicitly given.

Both () and [] are valid for indicated ranges, but only tuples can be used if (value,) for lower limit.

In [10]:
s.select_range(ra=(114,))

### Quantities can be give as [v1,v2]\*unit or [v1\*unit,v2\*unit]

In [11]:
s.select_range(dec=[2400, 7500] * u.arcmin)
s.show()

 ID    TAG     CRVAL2     CRVAL3    # SELECTED
--- --------- ------- ------------- ----------
  0 2c4eece7b [114.0]                       40
  1 7cd556e4e         [40.0, 125.0]         30


In [12]:
s.final

,OBJECT,BANDWID,DATE-OBS,DURATION,EXPOSURE,TSYS,TDIM7,TUNIT7,CTYPE1,CRVAL1,...,FDNUM,INT,NSAVE,HDU,BINTABLE,ROW,PROC,_OBSTYPE,_SUBOBSMODE,UTC
0,NGC2782,23437500.0,2021-02-10T07:58:44.98,1.999285,1.976916,1.0,"(32768,1,1,1)",Counts,FREQ-OBS,1.408344e+09,...,0,16,-1,1,0,20,OnOff,PSWITCHON,TPWCAL,2021-02-10T07:57:40.000
1,NGC2782,23437500.0,2021-02-10T07:58:44.98,1.999285,1.976916,1.0,"(32768,1,1,1)",Counts,FREQ-OBS,1.408344e+09,...,0,16,-1,1,0,21,OnOff,PSWITCHON,TPWCAL,2021-02-10T07:57:40.000
2,NGC2782,23437500.0,2021-02-10T07:58:44.98,1.999285,1.976916,1.0,"(32768,1,1,1)",Counts,FREQ-OBS,1.408344e+09,...,0,16,-1,1,0,22,OnOff,PSWITCHON,TPWCAL,2021-02-10T07:57:40.000
3,NGC2782,23437500.0,2021-02-10T07:58:44.98,1.999285,1.976916,1.0,"(32768,1,1,1)",Counts,FREQ-OBS,1.600239e+09,...,0,16,-1,1,0,23,OnOff,PSWITCHON,TPWCAL,2021-02-10T07:57:40.000
4,NGC2782,23437500.0,2021-02-10T07:58:44.98,1.999285,1.976916,1.0,"(32768,1,1,1)",Counts,FREQ-OBS,1.600239e+09,...,0,16,-1,1,0,24,OnOff,PSWITCHON,TPWCAL,2021-02-10T07:57:40.000
5,NGC2782,23437500.0,2021-02-10T07:58:44.98,1.999285,1.976916,1.0,"(32768,1,1,1)",Counts,FREQ-OBS,1.600239e+09,...,0,16,-1,1,0,25,OnOff,PSWITCHON,TPWCAL,2021-02-10T07:57:40.000
6,NGC2782,23437500.0,2021-02-10T07:58:44.98,1.999285,1.976916,1.0,"(32768,1,1,1)",Counts,FREQ-OBS,1.600239e+09,...,0,16,-1,1,0,26,OnOff,PSWITCHON,TPWCAL,2021-02-10T07:57:40.000
7,NGC2782,23437500.0,2021-02-10T07:58:44.98,1.999285,1.976916,1.0,"(32768,1,1,1)",Counts,FREQ-OBS,1.653339e+09,...,0,16,-1,1,0,27,OnOff,PSWITCHON,TPWCAL,2021-02-10T07:57:40.000
8,NGC2782,23437500.0,2021-02-10T07:58:44.98,1.999285,1.976916,1.0,"(32768,1,1,1)",Counts,FREQ-OBS,1.653339e+09,...,0,16,-1,1,0,28,OnOff,PSWITCHON,TPWCAL,2021-02-10T07:57:40.000
9,NGC2782,23437500.0,2021-02-10T07:58:44.98,1.999285,1.976916,1.0,"(32768,1,1,1)",Counts,FREQ-OBS,1.653339e+09,...,0,16,-1,1,0,29,OnOff,PSWITCHON,TPWCAL,2021-02-10T07:57:40.000


### Selection keywords are case insensitive
Note also elevation is aliased here to elevatio (the actual SDFITS keyword)

In [13]:
s = Selection(sdf)
s.select_within(eLEVaTIon=(18.0, 2))
s.show()
s.final

 ID    TAG      ELEVATIO   # SELECTED
--- --------- ------------ ----------
  0 362765ba3 [16.0, 20.0]         13


,OBJECT,BANDWID,DATE-OBS,DURATION,EXPOSURE,TSYS,TDIM7,TUNIT7,CTYPE1,CRVAL1,...,FDNUM,INT,NSAVE,HDU,BINTABLE,ROW,PROC,_OBSTYPE,_SUBOBSMODE,UTC
0,PSR_B0329+54,23437500.0,2021-02-10T07:15:16.00,0.081090,0.081090,1.0,"(32768,1,1,1)",Counts,FREQ-OBS,1.420207e+09,...,0,0,-1,1,0,0,Track,NONE,TPNOCAL,2021-02-10T07:15:16.000
1,PSR_B0329+54,23437500.0,2021-02-10T07:15:16.00,0.081090,0.081090,1.0,"(32768,1,1,1)",Counts,FREQ-OBS,1.612101e+09,...,0,0,-1,1,0,1,Track,NONE,TPNOCAL,2021-02-10T07:15:16.000
2,PSR_B0329+54,23437500.0,2021-02-10T07:15:16.00,0.081090,0.081090,1.0,"(32768,1,1,1)",Counts,FREQ-OBS,1.612101e+09,...,0,0,-1,1,0,2,Track,NONE,TPNOCAL,2021-02-10T07:15:16.000
3,PSR_B0329+54,23437500.0,2021-02-10T07:15:16.00,0.081090,0.081090,1.0,"(32768,1,1,1)",Counts,FREQ-OBS,1.665202e+09,...,0,0,-1,1,0,3,Track,NONE,TPNOCAL,2021-02-10T07:15:16.000
4,PSR_B0329+54,23437500.0,2021-02-10T07:15:16.00,0.081090,0.081090,1.0,"(32768,1,1,1)",Counts,FREQ-OBS,1.665202e+09,...,0,0,-1,1,0,4,Track,NONE,TPNOCAL,2021-02-10T07:15:16.000
5,PSR_B0329+54,23437500.0,2021-02-10T07:15:16.00,0.081090,0.081090,1.0,"(32768,1,1,1)",Counts,FREQ-OBS,1.667216e+09,...,0,0,-1,1,0,5,Track,NONE,TPNOCAL,2021-02-10T07:15:16.000
6,PSR_B0329+54,23437500.0,2021-02-10T07:15:16.00,0.081090,0.081090,1.0,"(32768,1,1,1)",Counts,FREQ-OBS,1.667216e+09,...,0,0,-1,1,0,6,Track,NONE,TPNOCAL,2021-02-10T07:15:16.000
7,PSR_B0329+54,23437500.0,2021-02-10T07:15:16.00,0.081090,0.081090,1.0,"(32768,1,1,1)",Counts,FREQ-OBS,1.720500e+09,...,0,0,-1,1,0,7,Track,NONE,TPNOCAL,2021-02-10T07:15:16.000
8,PSR_B0329+54,23437500.0,2021-02-10T07:15:16.00,0.081090,0.081090,1.0,"(32768,1,1,1)",Counts,FREQ-OBS,1.720500e+09,...,0,0,-1,1,0,8,Track,NONE,TPNOCAL,2021-02-10T07:15:16.000
9,PSR_B0329+54,23437500.0,2021-02-10T07:15:16.08,0.081090,0.081090,1.0,"(32768,1,1,1)",Counts,FREQ-OBS,1.420207e+09,...,0,1,-1,1,0,9,Track,NONE,TPNOCAL,2021-02-10T07:15:16.000


### Other aliases are pre-defined and the user can define more.

In [14]:
s.aliases

{'FREQ': 'CRVAL1',
 'RA': 'CRVAL2',
 'DEC': 'CRVAL3',
 'GLON': 'CRVAL2',
 'GLAT': 'CRVAL3',
 'GALLON': 'CRVAL2',
 'GALLAT': 'CRVAL3',
 'ELEVATION': 'ELEVATIO',
 'SOURCE': 'OBJECT',
 'POL': 'PLNUM'}

In [15]:
s.alias({'target':'object','az':'azimuth'})

In [16]:
s.aliases

{'FREQ': 'CRVAL1',
 'RA': 'CRVAL2',
 'DEC': 'CRVAL3',
 'GLON': 'CRVAL2',
 'GLAT': 'CRVAL3',
 'GALLON': 'CRVAL2',
 'GALLAT': 'CRVAL3',
 'ELEVATION': 'ELEVATIO',
 'SOURCE': 'OBJECT',
 'POL': 'PLNUM',
 'TARGET': 'OBJECT',
 'AZ': 'AZIMUTH'}

In [17]:
s.select(target='ScoX-1')

In [18]:
s.show()

 ID    TAG    OBJECT   ELEVATIO   # SELECTED
--- --------- ------ ------------ ----------
  0 362765ba3        [16.0, 20.0]         13
  1 17468954b ScoX-1                      10


### A selection which results in no data is checked for and is ignored.

In [19]:
s.select(target='foobar')

/home/mpound/src/dysh/src/dysh/util/selection.py:506: UserWarning: Your selection rule resulted in no data being selected. Ignoring.
  warnings.warn("Your selection rule resulted in no data being selected. Ignoring.")


### UTC time ranges can be selected with Time objects.
This checks against the UTC timestamp column.  For LST, use select_range(lst=[number1,number2])



In [20]:
s = Selection(sdf)
s.select_range(utc=(Time("2021-02-10T08:00", scale="utc"), 
                    Time("2021-02-10T09:00", scale="utc")))

In [21]:
s.final

,OBJECT,BANDWID,DATE-OBS,DURATION,EXPOSURE,TSYS,TDIM7,TUNIT7,CTYPE1,CRVAL1,...,FDNUM,INT,NSAVE,HDU,BINTABLE,ROW,PROC,_OBSTYPE,_SUBOBSMODE,UTC
0,ScoX-1,23437500.0,2021-02-10T08:48:35.98,0.499122,0.000000,1.0,"(32768,1,1,1)",Counts,FREQ-OBS,6.250564e+09,...,0,10,-1,1,0,40,OnOff,PSWITCHON,TPWCAL,2021-02-10T08:48:26.000
1,ScoX-1,23437500.0,2021-02-10T08:48:35.98,0.499122,0.040545,1.0,"(32768,1,1,1)",Counts,FREQ-OBS,6.250564e+09,...,0,10,-1,1,0,41,OnOff,PSWITCHON,TPWCAL,2021-02-10T08:48:26.000
2,ScoX-1,23437500.0,2021-02-10T08:48:35.98,0.499122,0.000000,1.0,"(32768,1,1,1)",Counts,FREQ-OBS,6.250564e+09,...,0,10,-1,1,0,42,OnOff,PSWITCHON,TPWCAL,2021-02-10T08:48:26.000
3,ScoX-1,23437500.0,2021-02-10T08:48:58.50,0.499122,0.487937,1.0,"(32768,1,1,1)",Counts,FREQ-OBS,4.500563e+09,...,0,0,-1,1,0,43,OnOff,PSWITCHOFF,TPWCAL,2021-02-10T08:48:58.000
4,ScoX-1,23437500.0,2021-02-10T08:48:58.50,0.499122,0.487937,1.0,"(32768,1,1,1)",Counts,FREQ-OBS,4.500563e+09,...,0,0,-1,1,0,44,OnOff,PSWITCHOFF,TPWCAL,2021-02-10T08:48:58.000
5,ScoX-1,23437500.0,2021-02-10T08:48:58.50,0.499122,0.487937,1.0,"(32768,1,1,1)",Counts,FREQ-OBS,4.500563e+09,...,0,0,-1,1,0,45,OnOff,PSWITCHOFF,TPWCAL,2021-02-10T08:48:58.000
6,ScoX-1,23437500.0,2021-02-10T08:48:58.50,0.499122,0.487937,1.0,"(32768,1,1,1)",Counts,FREQ-OBS,4.500563e+09,...,0,0,-1,1,0,46,OnOff,PSWITCHOFF,TPWCAL,2021-02-10T08:48:58.000
7,ScoX-1,23437500.0,2021-02-10T08:48:58.50,0.499122,0.487937,1.0,"(32768,1,1,1)",Counts,FREQ-OBS,4.750685e+09,...,0,0,-1,1,0,47,OnOff,PSWITCHOFF,TPWCAL,2021-02-10T08:48:58.000
8,ScoX-1,23437500.0,2021-02-10T08:48:58.50,0.499122,0.487937,1.0,"(32768,1,1,1)",Counts,FREQ-OBS,4.750685e+09,...,0,0,-1,1,0,48,OnOff,PSWITCHOFF,TPWCAL,2021-02-10T08:48:58.000
9,ScoX-1,23437500.0,2021-02-10T08:48:58.50,0.499122,0.487937,1.0,"(32768,1,1,1)",Counts,FREQ-OBS,4.750685e+09,...,0,0,-1,1,0,49,OnOff,PSWITCHOFF,TPWCAL,2021-02-10T08:48:58.000


### Channel selection has it's own method
Channels can be ranges, individual channels or combinations there of.
Note that selecting channels does not down select rows.

In [22]:
s = Selection(sdf)
a = [1, 4, (30, 40)]
s.select_channel(a)
s.show()

 ID    TAG          CHAN       # SELECTED
--- --------- ---------------- ----------
  0 c1733960c [1, 4, (30, 40)]         50


### You can only have one channel selection rule

In [23]:
try: 
    s.select_channel([60,70])
except Exception as e:
    print(e)

You can only have one channel selection rule. Remove the old rule before creating a new one.
